In [18]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from skimage.feature import peak_local_max
import cv2
from os import listdir
from os.path import isfile, join
import json
import pandas as pd
import time

# EXTRACTION MOMENTS OF TIME FOR WHICH THE DATA IS LABELED

In [47]:
''' EXTRACT TIME RANGE '''
mypath = 'CalculateDetectionMetrics/cropped_data_labeled/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
time_range = np.array(sorted([float(x[19:-5]) for x in list(filter(lambda s: 'json' in s, onlyfiles))]))
time_range = time_range[time_range >= 30]
time_range = np.round(time_range, 1)
# time_range = time_range[30]
time_range

array([ 30. ,  33. ,  36. ,  39. ,  42.1,  45. ,  48. ,  51. ,  54. ,
        57. ,  60. ,  63. ,  66. ,  69. ,  72. ,  75. ,  78. ,  81. ,
        84. ,  87. ,  90. ,  93. ,  96. ,  99. , 102. , 105. , 108. ,
       111. , 114. , 117. , 120. , 123. , 126. , 129. , 132. , 135. ,
       138. , 141. , 144. , 147. , 150. , 153. , 156. , 159. , 162. ,
       165. , 168. , 171. , 174. , 177. , 180. , 183. , 186. , 189. ,
       192. , 195. , 198. , 201. , 204. , 207. , 210.1, 214. , 217. ,
       220. , 223. , 226. , 229. , 232. , 235. , 238. , 241. , 244. ,
       247. , 250. , 253. , 256. , 259. , 262. , 265. , 268. , 271. ,
       274. , 277. , 280. , 283. , 286. , 289. , 292. , 295. , 298. ,
       301. , 304. , 307. , 310. , 313. , 316. , 319. , 322. , 325. ,
       328. ])

# BOHMIAN POTENTIAL METHOD ALGORITHM

In [42]:
def convert_coordinates_to_frame(mask, frame, coordinates):
    vortices_list = []
    for x,y in coordinates:
        cx= int(x * frame.shape[0] / mask.shape[0])
        cy= int(y * frame.shape[0] / mask.shape[0])
        vortices_list.append((cx, cy))
    return vortices_list

In [43]:
def difference(current, neighbor, value):
    if current - neighbor < value:
        return True
    else:
        return False

def create_range(x):
    param = 10
    return [a for a in range(x-(param-1), x) if a >= 0] + [x] + [a for a in range(x+1, x+param) if a <= 815]

In [48]:
program_starts = time.time()
detected_vortices = {}
N = 817
for t in time_range:
    Index = list(range(N))
    Cols = list(range(N))
    df = pd.read_csv(f'input_data/bohmian_potential/Bohmian_potential_time={np.round(t,1)}0.dat', delimiter='\t', header=0, index_col=0)
    df_xvel = pd.read_csv(f'input_data/x_vel/x_phase_velocity_snapshot_time={np.round(t,1)}0.dat', delimiter='\t', header=0, index_col=0)
    frame = cv2.imread(f'CalculateDetectionMetrics/cropped_data_labeled/x_vel_cropped_time={np.round(t, 1)}0.png')
    
    max_index = 816
    df = df.iloc[0:max_index, 0:max_index]
    df_xvel = df_xvel.iloc[0:max_index, 0:max_index]
    
    df = pd.DataFrame(data=np.rot90(df.values, 1), index= df.index, columns=df.columns)
    df_xvel = pd.DataFrame(data=np.rot90(df_xvel.values, 1), index= df_xvel.index, columns=df_xvel.columns)
    threshold = -1.0

    minima_coordinates = {}

    diff_value = -10

    
    for x in range(1, max_index-1):
        for y in range(1, max_index-1):
            cur = df.iloc[y, x]
            left = df.iloc[y-1, x]
            right = df.iloc[y+1, x]
            top = df.iloc[y, x+1]
            bottom = df.iloc[y, x-1]
            left_top = df.iloc[y-1, x+1]
            left_bottom = df.iloc[y-1, x-1]
            right_top = df.iloc[y+1, x+1]
            right_bottom = df.iloc[y+1, x-1]
            if (difference(cur, left, diff_value) and difference(cur, right, diff_value) and difference(cur, top, diff_value) and difference(cur, bottom, diff_value) and difference(cur, right_bottom, diff_value) and difference(cur, left_bottom, diff_value) and difference(cur, right_top, diff_value) and difference(cur, left_top, diff_value)):
                if cur < threshold:
                    minima_coordinates[(x, y)] = cur


    ''' EXTENDED NEIGHBORHOOD MODEL FILTERING '''
    minima_candidates = list(minima_coordinates.keys())
    final_minima = []
    for x, y in minima_candidates:
        # if not (x - 5 < 0 or y - 5 < 0 or x + 5 > 815 or y + 5 > (max_index-1)):
        x_range = create_range(x)
        y_range = create_range(y)
        find = False
        for y_new in [y-1, y, y+1]:
            for x_new in x_range:
                if df.iloc[y, x] > df.iloc[y_new, x_new]:
                    find = True
                    break
            if find:
                break
        if not find:
            for x_new in [x - 1, x, x + 1]:
                for y_new in y_range:
                    if df.iloc[y, x] > df.iloc[y_new, x_new]:
                        find = True
                        break
                if find:
                    break
        if not find:
            final_minima.append((x, y))
            
    '''GET DETECTED VORTICES COORDINATES'''
    detected_vortices[np.round(t,1)]  = convert_coordinates_to_frame(df.values, frame, final_minima)
    
now = time.time()
print("It has been {0} seconds since the loop started".format(now - program_starts))

It has been 9870.185135364532 seconds since the loop started


In [12]:
detected_vortices[np.round(t,1)]  = convert_coordinates_to_frame(df.values, frame, final_minima)

# TP, FP, FN METRICS CALCULATIONS

In [55]:
''' READ JSON FILES WITH LABELED BOUNDING BOXES '''
bbox_size = 3
for index, t in enumerate(time_range):
    with open(f'CalculateDetectionMetrics/cropped_data_labeled/x_vel_cropped_time={np.round(t,1)}0.json', 'r') as f:
        dictData = json.load(f)
    frame = cv2.imread(f'CalculateDetectionMetrics/cropped_data_labeled/x_vel_cropped_time={np.round(t,1)}0.png')
    TP = []
    for i, vortex in enumerate(dictData['shapes']):
        labeled_mask = np.zeros((frame.shape[:2]), dtype='uint8')
        a = int(vortex['points'][0][0])
        b = int(vortex['points'][0][1])
        c = int(vortex['points'][1][0])
        d = int(vortex['points'][1][1])
    #         print(f'vortex {i+1}', a, b, c, d)
        cv2.rectangle(labeled_mask, (a,b), (c,d), 255, cv2.FILLED)
        for x,y in detected_vortices[np.round(t,1)]:
            if (x,y) not in TP:
                detected_vortex = np.zeros((frame.shape[:2]), dtype='uint8')
                cv2.rectangle(detected_vortex, (x-bbox_size,y-bbox_size), (x+bbox_size,y+bbox_size), 255, cv2.FILLED)
                result = np.all(cv2.bitwise_and(detected_vortex, labeled_mask) == 0)
                if not result:
                    TP.append((x,y))
                    break
    tp_count = len(TP) #CORRECT
    fp_count = len(detected_vortices[np.round(t,1)]) - len(TP) #INCORRECT
    fn_count = len(dictData['shapes']) - len(TP) #UNDETECTED
    
    ''' INSPECTION IF CALCULATION WORK PROPERLY '''
    if index % 10 == 0:
        for (x,y) in detected_vortices[np.round(t,1)]:
            if (x,y) in TP:
                color = (0,255,0)
            else:
                color = (0,0,255)
            cv2.rectangle(frame, (x-12,y-12), (x+12,y+12), color, 2)
        for parameter, value, shift in zip(['T=', 'TP=', 'FP=', 'FN='], [t, tp_count, fp_count, fn_count], [30, 60, 90, 120]):
            cv2.putText(frame, text=parameter+str(value), org=(10, shift),
                    fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.9, color=(0,0,0),
                    thickness=1, lineType=cv2.LINE_AA)
        cv2.imwrite(f'Straight-Forward-BP_Method-Find_Local_Minima_in_2D_Array_RESULTS/TP_FP_{np.round(t,1)}.png', frame)
        print(t)
        
    ''' SAVE RESULTS TO FILE '''
#     print(t, tp_count, fp_count, fn_count)
    with open('Straight-Forward-BP_Method-Find_Local_Minima_in_2D_Array_RESULTS/Straight-Forward-BPMethod-Detection-Metrics-Results.csv', 'a') as fr:
        fr.write(f'{t}\t{tp_count}\t{fp_count}\t{fn_count}\n')

30.0
60.0
90.0
120.0
150.0
180.0
210.1
241.0
271.0
301.0


# FINAL METRICS SUMMATION

In [56]:
df_metrics = pd.read_csv('Straight-Forward-BP_Method-Find_Local_Minima_in_2D_Array_RESULTS/Straight-Forward-BPMethod-Detection-Metrics-Results.csv', header=None, delimiter='\t')
df_metrics.columns = ['TIME', 'TP', 'FP', 'FN']
df_metrics.set_index('TIME', inplace=True)
df_metrics.sum()

TP    2170
FP     154
FN     183
dtype: int64

In [54]:
detected_vortices

{30.0: [(427, 725), (519, 817), (724, 241), (1004, 521)],
 33.0: [(521, 395), (747, 30), (849, 724), (1214, 498)],
 36.0: [(24, 531),
  (704, 215),
  (713, 1221),
  (732, 391),
  (854, 513),
  (1029, 540)],
 39.0: [(394, 534),
  (423, 735),
  (510, 822),
  (522, 192),
  (710, 851),
  (1052, 722)],
 42.1: [(750, 463), (782, 495)],
 45.0: [(385, 524),
  (400, 534),
  (403, 568),
  (514, 421),
  (676, 842),
  (681, 136),
  (710, 845),
  (721, 860),
  (823, 730),
  (1109, 563)],
 48.0: [],
 51.0: [(437, 761), (484, 808)],
 54.0: [(481, 447),
  (498, 348),
  (560, 48),
  (693, 44),
  (797, 764),
  (897, 747),
  (1196, 684),
  (1201, 551)],
 57.0: [(357, 530), (493, 420), (715, 887), (825, 751)],
 60.0: [(461, 459),
  (470, 417),
  (692, 32),
  (785, 783),
  (828, 774),
  (1213, 553)],
 63.0: [(327, 649),
  (334, 719),
  (377, 518),
  (461, 389),
  (525, 910),
  (595, 918),
  (727, 868),
  (855, 783)],
 66.0: [(285, 704),
  (302, 686),
  (305, 493),
  (415, 484),
  (505, 441),
  (540, 959),


# SAVE TO FILE DETECTED COORDINATES

In [53]:
with open('Straight-Forward-BP_Method-Find_Local_Minima_in_2D_Array_RESULTS/detected_vortices.txt','w') as data: 
      data.write(str(detected_vortices))